In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

## Handwritten Numerical Digit Classification

For this demonstration, we will be using a pre-loaded dataset in torchvision called MNIST. This is a large database of labeled handwritten digits. We will build a Nueral Network using PyTorch that will classify these hadwritting images.

In [2]:
# Importing our MNIST Data:

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values to [-1, 1]
])

# Download and load the training set
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test set
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

This is the basic structure of the Neural Network. Each image is 28x28 pixels, so our input layer must be 28x28. I created an input layer, a hidden layer, and an output layer. The output layer must have 10 classes, as there are 10 possible classifications

In [3]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.p1 = nn.Linear(28*28, 128)  # (28*28 pixels -> 128 nodes in hidden layer)
        self.p2 = nn.Linear(128, 10)     # (128 nodes -> 10 nodes)

    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the input images
        x = torch.relu(self.p1(x))  # Apply ReLU activation to hidden layer
        x = self.p2(x)  # Output layer
        return x


In [4]:
# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Train the model
epochs = 4
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")

print("Training finished!")

# Save the trained model
torch.save(model.state_dict(), "simple_nn_model.pth")

Epoch 1/4, Loss: 0.4438566872830203
Epoch 2/4, Loss: 0.23481655820235134
Epoch 3/4, Loss: 0.17642708073086194
Epoch 4/4, Loss: 0.14266741420946585
Training finished!


In [5]:
# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {100 * accuracy:.2f}%")

Accuracy on the test set: 95.70%
